In [2]:
import numpy as np

from reachy_sdk import ReachySDK

from reachy_sdk.trajectory import goto
from reachy_sdk.trajectory.interpolation import InterpolationMode

In [3]:
reachy = ReachySDK('localhost')

Check if all the joints seem to be detected.

In [3]:
reachy

<Reachy host="localhost" joints=
	<Joint name="l_shoulder_pitch" pos="-14.75" mode="compliant">
	<Joint name="l_shoulder_roll" pos="-11.45" mode="compliant">
	<Joint name="l_arm_yaw" pos="-1.27" mode="compliant">
	<Joint name="l_elbow_pitch" pos="-61.58" mode="compliant">
	<Joint name="l_forearm_yaw" pos="-70.23" mode="compliant">
	<Joint name="l_wrist_pitch" pos="51.38" mode="compliant">
	<Joint name="l_wrist_roll" pos="-54.69" mode="compliant">
	<Joint name="l_gripper" pos="30.28" mode="compliant">
	<Joint name="r_shoulder_pitch" pos="10.92" mode="compliant">
	<Joint name="r_shoulder_roll" pos="2.13" mode="compliant">
	<Joint name="r_arm_yaw" pos="12.35" mode="compliant">
	<Joint name="r_elbow_pitch" pos="-62.55" mode="compliant">
	<Joint name="r_forearm_yaw" pos="12.76" mode="compliant">
	<Joint name="r_wrist_pitch" pos="-32.13" mode="compliant">
	<Joint name="r_wrist_roll" pos="-8.06" mode="compliant">
	<Joint name="r_gripper" pos="-33.80" mode="compliant">
	<Joint name="l_antenna"

## Checking compliance

In [4]:
reachy.turn_on('reachy')

In [5]:
reachy.turn_off('reachy')

Lift both arms, put them in stiff mode and check if *turn_off_smoothly* works.

In [6]:
reachy.turn_on('reachy')

In [4]:
reachy.turn_off_smoothly('reachy')

## Moving the arms

Put each arm at 90 degrees.

In [8]:
try:
    right_angled_position_right = {
    reachy.r_arm.r_shoulder_pitch: 0,
    reachy.r_arm.r_shoulder_roll: 0,
    reachy.r_arm.r_arm_yaw: 0,
    reachy.r_arm.r_elbow_pitch: -90,
    reachy.r_arm.r_forearm_yaw: 0,
    reachy.r_arm.r_wrist_pitch: 0,
    reachy.r_arm.r_wrist_roll: 0,
}
except AttributeError:
    pass

try:
    right_angled_position_left = {
        reachy.l_arm.l_shoulder_pitch: 0,
        reachy.l_arm.l_shoulder_roll: 0,
        reachy.l_arm.l_arm_yaw: 0,
        reachy.l_arm.l_elbow_pitch: -90,
        reachy.l_arm.l_forearm_yaw: 0,
        reachy.l_arm.l_wrist_pitch: 0,
        reachy.l_arm.l_wrist_roll: 0,
    }
except AttributeError:
    pass

In [19]:
reachy.turn_on('reachy')

In [10]:
try:
    goto(
        goal_positions=right_angled_position_right,
        duration=1.0,
        interpolation_mode=InterpolationMode.MINIMUM_JERK
    )
except NameError:
    print('Reachy has no right arm!')

In [11]:
try:
    goto(
        goal_positions=right_angled_position_left,
        duration=1.0,
        interpolation_mode=InterpolationMode.MINIMUM_JERK
    )
except NameError:
    print('Reachy has no left arm!')

In [1]:
reachy.turn_off_smoothly('reachy')

Reproduce the square movement from [the documentation](https://pollen-robotics.github.io/reachy-2021-docs/sdk/first-moves/kinematics/#inverse-kinematics), to check if this Reachy can reproduce it.

For the right arm:

In [13]:
A = np.array([
  [0, 0, -1, 0.3],
  [0, 1, 0, -0.4],  
  [1, 0, 0, -0.3],
  [0, 0, 0, 1],  
])

B = np.array([
  [0, 0, -1, 0.3],
  [0, 1, 0, -0.4],  
  [1, 0, 0, 0.0],
  [0, 0, 0, 1],  
])

C = np.array([
  [0, 0, -1, 0.3],
  [0, 1, 0, -0.1],  
  [1, 0, 0, 0.0],
  [0, 0, 0, 1],  
])

D = np.array([
  [0, 0, -1, 0.3],
  [0, 1, 0, -0.1],  
  [1, 0, 0, -0.3],
  [0, 0, 0, 1],  
])

In [14]:
try:
    joint_pos_A = reachy.r_arm.inverse_kinematics(A)
    joint_pos_B = reachy.r_arm.inverse_kinematics(B)
    joint_pos_C = reachy.r_arm.inverse_kinematics(C)
    joint_pos_D = reachy.r_arm.inverse_kinematics(D)
except AttributeError:
    print('Reachy has no right arm!')

In [15]:
import time
# put the joints in stiff mode
try:
    reachy.turn_on('r_arm')

    # use the goto function
    goto({joint: pos for joint,pos in zip(reachy.r_arm.joints.values(), joint_pos_A)}, duration=1.0)
    time.sleep(0.5)
    goto({joint: pos for joint,pos in zip(reachy.r_arm.joints.values(), joint_pos_B)}, duration=1.0)
    time.sleep(0.5)
    goto({joint: pos for joint,pos in zip(reachy.r_arm.joints.values(), joint_pos_C)}, duration=1.0)
    time.sleep(0.5)
    goto({joint: pos for joint,pos in zip(reachy.r_arm.joints.values(), joint_pos_D)}, duration=1.0)

    # put the joints back to compliant mode
    # use turn_off_smoothly to prevent the arm from falling hard
    reachy.turn_off_smoothly('r_arm')
    
except AttributeError:
    print('Reachy has no right arm!')

For the left arm:

In [20]:
A = np.array([
  [0, 0, -1, 0.3],
  [0, 1, 0, 0.1],  
  [1, 0, 0, -0.3],
  [0, 0, 0, 1],  
])

B = np.array([
  [0, 0, -1, 0.3],
  [0, 1, 0, 0.1],  
  [1, 0, 0, 0.0],
  [0, 0, 0, 1],  
])

C = np.array([
  [0, 0, -1, 0.3],
  [0, 1, 0, 0.3],  
  [1, 0, 0, 0.0],
  [0, 0, 0, 1],  
])

D = np.array([
  [0, 0, -1, 0.3],
  [0, 1, 0, 0.3],  
  [1, 0, 0, -0.3],
  [0, 0, 0, 1],  
])

In [18]:
try:
    joint_pos_A = reachy.l_arm.inverse_kinematics(A)
    joint_pos_B = reachy.l_arm.inverse_kinematics(B)
    joint_pos_C = reachy.l_arm.inverse_kinematics(C)
    joint_pos_D = reachy.l_arm.inverse_kinematics(D)
except AttributeError:
    print('Reachy has no left arm')

In [21]:
# put the joints in stiff mode
try:
    reachy.turn_on('l_arm')

    # use the goto function
    goto({joint: pos for joint,pos in zip(reachy.l_arm.joints.values(), joint_pos_A)}, duration=1.0)
    time.sleep(0.5)
    goto({joint: pos for joint,pos in zip(reachy.l_arm.joints.values(), joint_pos_B)}, duration=1.0)
    time.sleep(0.5)
    goto({joint: pos for joint,pos in zip(reachy.l_arm.joints.values(), joint_pos_C)}, duration=1.0)
    time.sleep(0.5)
    goto({joint: pos for joint,pos in zip(reachy.l_arm.joints.values(), joint_pos_D)}, duration=1.0)

    # put the joints back to compliant mode
    # use turn_off_smoothly to prevent the arm from falling hard
    reachy.turn_off_smoothly('l_arm')
except AttributeError:
    print('Reachy has no left arm!')

## Checking the force sensors

You should see one force sensor per arm, zith the *.force_sensors* object.

In [7]:
reachy.force_sensors

<Holder
	<ForceSensor name="l_force_gripper" force="2.21">
	<ForceSensor name="r_force_gripper" force="6.40">
>

The force value should be positive when applying a force towards the exterior of the gripper (e.g. when you try to open the gripper which is in stiff mode). On the contrary, you should get a negative force value when applying a force towards the interior of the gripper.  

First, place the grippers in stiff mode, it will be easier to check the force values.

In [5]:
try:
    reachy.l_arm.l_gripper.compliant = False
except AttributeError:
    print('Reachy has no left arm.')

In [6]:
try:
    reachy.r_arm.r_gripper.compliant = False
except AttributeError:
    print('Reachy has no right arm.')

Now, try to close each gripper and check that when you doing this, the force value is negative.

In [12]:
reachy.force_sensors.r_force_gripper

<ForceSensor name="r_force_gripper" force="-118.81">

In [13]:
reachy.force_sensors.l_force_gripper

<ForceSensor name="l_force_gripper" force="-133.14">

Next, try to open each gripper and check that when you doing this, the force value is positive.

In [14]:
reachy.force_sensors.r_force_gripper

<ForceSensor name="r_force_gripper" force="95.95">

In [15]:
reachy.force_sensors.l_force_gripper

<ForceSensor name="l_force_gripper" force="69.74">

Finally, put the grippers back to compliant mode.

In [21]:
try:
    reachy.l_arm.l_gripper.compliant = True
except AttributeError:
    pass

In [22]:
try:
    reachy.r_arm.r_gripper.compliant = True
except AttributeError:
    pass